In [13]:
import sqlite3 as sl

server = sl.connect('server.db')
cursor = server.cursor()

In [14]:
cursor.execute("SELECT * FROM acc;")
cursor.fetchall()

[(1, 'admin', 'pass', 'OLEG', 'admin@mail.ru', None, None, None, None, None)]

In [19]:
cursor.execute("SELECT * FROM projects;")
cursor.fetchall()

[(1,
  1,
  '{"name":"1234","instance":{"nodes":[{"width":150,"height":40,"id":"1684055703862","position":{"x":450,"y":150},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"inplace":{"Type":"<class \'bool\'>","Default":"False"}},"label":"ReLU"},"selected":false,"positionAbsolute":{"x":450,"y":150},"dragging":false},{"width":150,"height":40,"id":"1684055714461","position":{"x":450,"y":90},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"in_features":{"Type":"<class \'int\'>","Default":null},"out_features":{"Type":"<class \'int\'>","Default":null},"bias":{"Type":"<class \'bool\'>","Default":"True"},"device":{"Type":null,"Default":"None"},"dtype":{"Type":null,"Default":"None"}},"label":"Linear"},"selected":false,"positionAbsolute":{"x":450,"y":90},"dragging":false},{"width":150,"height":40,"id":"1684055717039","position":{"x":450,"y":15},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"in1_features":{"Type":"<class \'int\'>","Default":null},"in

In [17]:
s = '{"name":"123","instance":{"nodes":[{"width":150,"height":40,"id":"1684055703862","position":{"x":450,"y":150},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"inplace":{"Type":"<class \'bool\'>","Default":"False"}},"label":"ReLU"},"selected":false,"positionAbsolute":{"x":450,"y":150},"dragging":false},{"width":150,"height":40,"id":"1684055714461","position":{"x":450,"y":90},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"in_features":{"Type":"<class \'int\'>","Default":null},"out_features":{"Type":"<class \'int\'>","Default":null},"bias":{"Type":"<class \'bool\'>","Default":"True"},"device":{"Type":null,"Default":"None"},"dtype":{"Type":null,"Default":"None"}},"label":"Linear"},"selected":false,"positionAbsolute":{"x":450,"y":90},"dragging":false},{"width":150,"height":40,"id":"1684055717039","position":{"x":450,"y":15},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"in1_features":{"Type":"<class \'int\'>","Default":null},"in2_features":{"Type":"<class \'int\'>","Default":null},"out_features":{"Type":"<class \'int\'>","Default":null},"bias":{"Type":"<class \'bool\'>","Default":"True"},"device":{"Type":null,"Default":"None"},"dtype":{"Type":null,"Default":"None"}},"label":"Bilinear"},"selected":false,"positionAbsolute":{"x":450,"y":15},"dragging":false}],"edges":[{"source":"1684055717039","sourceHandle":null,"target":"1684055714461","targetHandle":null,"id":"reactflow__edge-1684055717039-1684055714461"},{"source":"1684055714461","sourceHandle":null,"target":"1684055703862","targetHandle":null,"id":"reactflow__edge-1684055714461-1684055703862"}],"viewport":{"x":-473.5,"y":103.29999999999998,"zoom":2}}}'
s

'{"name":"123","instance":{"nodes":[{"width":150,"height":40,"id":"1684055703862","position":{"x":450,"y":150},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"inplace":{"Type":"<class \'bool\'>","Default":"False"}},"label":"ReLU"},"selected":false,"positionAbsolute":{"x":450,"y":150},"dragging":false},{"width":150,"height":40,"id":"1684055714461","position":{"x":450,"y":90},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"in_features":{"Type":"<class \'int\'>","Default":null},"out_features":{"Type":"<class \'int\'>","Default":null},"bias":{"Type":"<class \'bool\'>","Default":"True"},"device":{"Type":null,"Default":"None"},"dtype":{"Type":null,"Default":"None"}},"label":"Linear"},"selected":false,"positionAbsolute":{"x":450,"y":90},"dragging":false},{"width":150,"height":40,"id":"1684055717039","position":{"x":450,"y":15},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"in1_features":{"Type":"<class \'int\'>","Default":null},"in2_features":{

In [18]:
query = "INSERT INTO projects(idUser, data, descr) VALUES (?, ?, ?)"
a = (1, s, None)
cursor.execute(query, a)
server.commit()

'{"name":"1234","instance":{"nodes":[{"width":150,"height":40,"id":"1684055703862","position":{"x":450,"y":150},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"inplace":{"Type":"<class \'bool\'>","Default":"False"}},"label":"ReLU"},"selected":false,"positionAbsolute":{"x":450,"y":150},"dragging":false},{"width":150,"height":40,"id":"1684055714461","position":{"x":450,"y":90},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"in_features":{"Type":"<class \'int\'>","Default":null},"out_features":{"Type":"<class \'int\'>","Default":null},"bias":{"Type":"<class \'bool\'>","Default":"True"},"device":{"Type":null,"Default":"None"},"dtype":{"Type":null,"Default":"None"}},"label":"Linear"},"selected":false,"positionAbsolute":{"x":450,"y":90},"dragging":false},{"width":150,"height":40,"id":"1684055717039","position":{"x":450,"y":15},"data":{"Doc":null,"Args":{"self":{"Type":null,"Default":null},"in1_features":{"Type":"<class \'int\'>","Default":null},"in2_features":

In [2]:
query = """
CREATE TABLE IF NOT EXISTS acc(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    login TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL,
    username TEXT UNIQUE NOT NULL,
    mail TEXT UNIQUE NOT NULL,
    reg_date TEXT,
    first_name TEXT,
    last_name TEXT,
    graduation TEXT,
    company TEXT
)
"""
cursor.execute(query)
server.commit()

In [3]:
cursor.execute(
    """
    INSERT INTO acc(login, password, username, mail)
    VALUES('admin', 'pass', 'OLEG', 'admin@mail.ru')
    """
)
server.commit()

In [4]:
cursor.execute("SELECT * FROM acc;")
cursor.fetchall()

[(1, 'admin', 'pass', 'OLEG', 'admin@mail.ru', None, None, None, None, None)]

In [7]:
query = """
CREATE TABLE IF NOT EXISTS blocks(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    idUser INTEGER NOT NULL,
    data TEXT NOT NULL,
    descr TEXT,
    FOREIGN KEY(idUser) REFERENCES acc(id)
    ON DELETE CASCADE
    ON UPDATE CASCADE
)
"""
cursor.execute(query)
server.commit()

In [8]:
query = """
CREATE TABLE IF NOT EXISTS projects(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    idUser INTEGER NOT NULL,
    data TEXT NOT NULL,
    descr TEXT,
    FOREIGN KEY(idUser) REFERENCES acc(id)
    ON DELETE CASCADE
    ON UPDATE CASCADE
)
"""
cursor.execute(query)
server.commit()